#### Import

In [ ]:
import easyocr
import os
import torch
import matplotlib.pyplot as plt
import random
import glob
from tqdm import tqdm
import numpy as np
import time
from mmocr_eval import eval_ocr_metric
import re
import cv2 as cv

#### Install packges

In [ ]:
# !pip install matplotlib
# !pip install rapidfuzz

In [ ]:
torch.cuda.is_available()

#### Training

Load model by easyOCR API (https://www.jaided.ai/easyocr/documentation/)

In [ ]:
start = time.time()
model = 'crnn_30k_lr_25e-2'
iteration = 'iter_20000'

model_path = os.path.join('trainer', 'saved_models',model)
reader = easyocr.Reader(lang_list = ['en','th'],
                    model_storage_directory = model_path,
                    user_network_directory = os.path.join('my_model','user_network'),
                    recog_network  = iteration,
                    config_path = 'crnn',
                    gpu = True)
# reader = easyocr.Reader(lang_list = ['en','th'],gpu = True )
end = time.time()
print(f'loading model time: {end - start}')

visualise testing with some image

In [ ]:
list_im = [789, 782, 575, 779, 589, 714, 999 ,683]

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))

plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 1
rows = 1
fig = plt.figure(figsize=(15, 15))
ax = []
for i in tqdm(range(columns*rows)):
    # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

    # random_file_path = random.sample(filepaths,1)[0]
    random_file_path = filepaths[788]
    # random_file_path = filepaths[list_im.pop(0)]
    img = plt.imread(random_file_path)
    result = reader.recognize(random_file_path) 
    ax.append(fig.add_subplot(rows, columns, i+1) )
    boxs = []
    txt = []
    for idx,i in enumerate(result):
        boxs.append(i[0])
        txt.append(i[1])

    if len(result) > 0:
        ax[-1].set_title(f'found {idx+1}    predict: {txt}' , fontsize=20)
        for i in boxs:
            pts = np.array(i, np.int32)
            pts = pts.reshape((-1,1,2))
            cv.polylines(img,[pts],True,(0,0,0))

        
    else:
        ax[-1].set_title('No result', fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()

In [ ]:
# filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
# label_path = 'trainer/all_data/testing/testing/label.txt'
# with open(label_path, encoding='utf8') as file:
#     label = np.loadtxt(file,dtype=str)
# word_correct = 0

In [ ]:

# plt.rcParams['font.family'] = 'TH Sarabun New'
# columns = 5
# rows = 200
# fig, ax = plt.subplots(rows, columns, figsize=(20, 20))
# ax.set_facecolor("white")
# counter = 0
# for r in tqdm(range(rows)):
#     for c in range(columns):
#         file_name = re.split(r'[/\\]',filepaths[counter])[-1]
#         file_path = os.path.join('trainer/all_data/testing/testing',file_name)
#         img = plt.imread(file_path)
#         ax[r, c].axis('off')
#         result = reader.readtext(file_path) 
#         if len(result) > 0:
#             txt = ''
#             for i in result:
#                 txt += (' '+i[1])
#             ax[r, c].set_title(f'{file_name} predict: {txt}')
#         else:
#             ax[r, c].set_title('No result')
#         ax[r, c].imshow(img)
#         counter += 1
# plt.subplots_adjust(top = 20)

testing with all files in dataset

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))
label_path = 'trainer/all_data/testing/testing/label.txt'
with open(label_path, encoding='utf8') as file:
    label = np.loadtxt(file,dtype=str)

start = time.time()
# filepaths = filepaths[:10]
pred = []
gt = [] # ground truth
name_img = []
for i in tqdm(range(len(filepaths))):
    label_img = label[i][1]
    name_img.append(label[i][0])
    result = reader.recognize(filepaths[i], batch_size = 8,detail=0)
    try:
        pred.append(result)
    except:
        pred.append('')
    gt.append(label_img)
end = time.time()
print(f'time: {end - start}')

In [ ]:
import mmocr_eval 
from importlib import reload
reload(mmocr_eval)
tmp = name_img.copy()
output, false_list = mmocr_eval.eval_ocr_metric(pred, gt, tmp)
# output, false_list = eval_ocr_metric(pred, gt, name_img)
for k,v in output.items():
    print(f'{k}: {v}')

In [ ]:
# for i in false_list:
#     # print(i)

visualise testing with flase prediction

In [ ]:
filepaths = glob.glob(os.path.join('trainer/all_data/testing/testing','*.jpg'))
filepaths = sorted(filepaths, key= lambda x: int(re.split(r'[/\\.]',x)[-2]))

plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 3
rows = 5
fig = plt.figure(figsize=(15, 15))
ax = []
for i in tqdm(range(columns*rows)):
    # random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]

    random_file_name = random.sample(false_list,1)[0]
    random_file_name = random_file_name[0]
    random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)

    img = plt.imread(random_file_path)
    result = reader.recognize(random_file_path)
    ax.append(fig.add_subplot(rows, columns, i+1) )
    boxs = []
    txt = []
    for idx,i in enumerate(result):
        boxs.append(i[0])
        txt.append(i[1])


    if len(result) > 0:
        ax[-1].set_title(f'file: {random_file_name} predict: {txt} ', fontsize=20) 
        for i in boxs:
                pts = np.array(i, np.int32)
                pts = pts.reshape((-1,1,2))
                cv.polylines(img,[pts],True,(0,0,0))
    else:
        ax[-1].set_title(f'file: {random_file_name} No result', fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()

testing with LMWN

In [ ]:
plt.rcParams['font.family'] = 'TH Sarabun New'
plt.rcParams['xtick.labelsize'] = 20.0
plt.rcParams['ytick.labelsize'] = 20.0
columns = 2
rows = 10
fig = plt.figure(figsize=(10, 10))
ax = []
for i in range(columns*rows):
    random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/LMWN')),1)[0]
    # random_file_path = os.path.join('trainer/all_data/testing/testing',random_file_name)
    #random_file_name = random.sample(os.listdir(os.path.join('trainer/all_data/testing/testing')),1)[0]
    # random_file_name = os.listdir(os.path.join('trainer/all_data/testing/LMWN'))[2]
    random_file_path = os.path.join('trainer/all_data/testing/LMWN',random_file_name)
    img = plt.imread(random_file_path)
    result = reader.recognize(random_file_path) 
    ax.append(fig.add_subplot(rows, columns, i+1) )
    if len(result) > 0:
        ax[-1].set_title(f'predict: "{result[0][1]}" ', fontsize=20)
        # pts = np.array(result[0][0], np.int32)
        # pts = pts.reshape((-1,1,2))
        # cv.polylines(img,[pts],True,(255,0,0))
    else:
        ax[-1].set_title('No result',fontsize=20)
    plt.imshow(img)
fig.tight_layout() 
plt.show()